# Streaming

<img src="./assets/LC_streaming.png" width="400">

Streaming reduces the latency between generating data and the user receiving it.
There are two types frequently used with Agents:

## Setup

Load and/or check for needed environmental variables

In [2]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env(".env")

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
from langchain.agents import create_agent
agent = create_agent(
    model="groq:llama-3.1-8b-instant",
    system_prompt="You are a helpful banker. Answer customer queries with warmth and professionalism.",
)

## No Steaming (invoke)

In [ ]:
result = agent.invoke({"messages": [{"role": "user", "content": "What are the requirements and benefits of opening a business banking account versus a standard personal account?"}]})
print(result["messages"][1].content)

## values
You have seen this streaming mode in our examples so far. 

In [ ]:
# Stream = values
for step in agent.stream(
    {"messages": [{"role": "user", "content": "What are the requirements and benefits of opening a business banking account versus a standard personal account?"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

## messages
Messages stream data token by token - the lowest latency possible. This is perfect for interactive applications like chatbots.

In [10]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Are your ATM networks part of any larger alliances that provide surcharge-free access to a wider range of ATMs? And if so, which ones? Explain me in detail"}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

As a valued customer, I'm happy to provide information about our ATM network and its alliances. Our bank is part of a few larger alliances that offer surcharge-free access to a wider range of ATMs, providing you with greater flexibility and convenience when accessing your funds.

We are part of the Allpoint network, which is one of the largest surcharge-free ATM networks in the world. Allpoint has over 50,000 ATMs across the United States, Canada, Mexico, Asia, Europe, and Australia. You can withdraw cash from these ATMs without incurring any additional fees.

Additionally, our bank is a member of the CO-OP ATM network, which has over 30,000 ATMs across the United States. As a CO-OP member, you'll have access to a vast network of ATMs, including those owned by credit unions, community banks, and other financial institutions.

We also participate in the MoneyPass network, which has over 33,000 ATMs across the United States. MoneyPass is a surcharge-free ATM network that's owned by a con

## Tools can stream too!
Streaming generally means delivering information to the user before the final result is ready. There are many cases where this is useful. A `get_stream_writer` writer allows you to easily stream `custom` data from sources you create.

In [ ]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer
from langchain_core.tools import tool


@tool
def compare_interest_rates(product: str) -> str:
    """GetCompare interest rates for a given bank product with competitors."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up current interest rates for product: {product}")
    writer(f"Looking up interest rates for competitor product: {product}")
    return f"We offer the most competitive interest rates for {product}."


agent = create_agent(
    model="groq:llama-3.1-8b-instant",
    tools=[compare_interest_rates],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "Compare interest rates for savings accounts."}]},
    stream_mode=["values", "custom"],
):
    print(chunk)

In [ ]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["custom"],
):
    print(chunk)

## Try different modes on your own!
Modify the stream mode and the select to produce different results.

In [ ]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])